# Player Acquisition Models
*This notebook is used to train a machine learning model using scikit-learn and save it in ONNX format.*

## Library imports

In [ ]:
import logging
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

from skl2onnx import to_onnx
import onnxruntime as rt

## Configure logging

In [ ]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='player_acquisition_notebook.log',  
    level=logging.INFO,
)

## Load data

In [ ]:
dataset=pd.read_csv("player_training_data_full.csv")

## Prepare and split data

In [ ]:
X = dataset[['waiver_value_tier','fantasy_regular_season_weeks_remaining',
             'league_budget_pct_remaining']]
y = dataset['winning_bid_dollars']
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state = 0)

## Creating and fitting models

In [ ]:
model_10th_percentile = GradientBoostingRegressor(loss="quantile", alpha=0.1)
model_50th_percentile = GradientBoostingRegressor(loss="quantile", alpha=0.5)
model_90th_percentile = GradientBoostingRegressor(loss="quantile", alpha=0.9)

model_10th_percentile.fit(X_train, y_train)
model_50th_percentile.fit(X_train, y_train)
model_90th_percentile.fit(X_train, y_train)

## Save these models to ONNX format

In [ ]:
X_array = np.column_stack((X['waiver_value_tier'], 
                           X['fantasy_regular_season_weeks_remaining'], 
                           X['league_budget_pct_remaining']))

onx = to_onnx(model_10th_percentile, X_array[:1])
with open("acquisition_model_10.onnx", "wb") as f:
    f.write(onx.SerializeToString())

onx = to_onnx(model_50th_percentile, X_array[:1])
with open("acquisition_model_50.onnx", "wb") as f:
    f.write(onx.SerializeToString())    

onx = to_onnx(model_90th_percentile, X_array[:1])
with open("acquisition_model_90.onnx", "wb") as f:
    f.write(onx.SerializeToString())